In [6]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from pandas import read_csv
from sklearn.model_selection import train_test_split


file_name = "merged.csv"
df = pd.read_csv(file_name)

columns_to_keep = ['Current(A)', 'Voltage(V)', 'temperature', 'Step_Time(s)', 'Step_Index']
df = df[columns_to_keep]

df = df.dropna()

Qn = 2.0
initial_SOC = 0.8

df['SOC'] = 0.0


for i in range(len(df)):
    if i == 0 or df.loc[i, 'Step_Index'] != df.loc[i - 1, 'Step_Index']:
        df.loc[i, 'SOC'] = initial_SOC
    else:

        dt = (df.loc[i, 'Step_Time(s)'] - df.loc[i - 1, 'Step_Time(s)']) / 3600

        df.loc[i, 'SOC'] = (df.loc[i - 1, 'SOC'] + (df.loc[i, 'Current(A)'] / Qn) * dt).clip(0, 1)

output_file_name = "preprocessed_data.csv"
df.to_csv(output_file_name, index=False)

print(f"Preprocessing complete. Data saved to {output_file_name}.")



features_columns = ["Voltage(V)", "Current(A)", "temperature"]
target_column = ["SOC"]

dataset = df


features = dataset[features_columns]
targets = dataset[target_column]


X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)


initial_learning_rate = 0.01
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=1000, decay_rate=0.96, staircase=True
)


model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(features.shape[1],)),


    keras.layers.BatchNormalization(),

        keras.layers.Dense(16, activation='relu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),

        keras.layers.Dense(8, activation='relu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),

        keras.layers.Dense(4, activation='relu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),




    keras.layers.Dense(1, activation='sigmoid')
])


model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='mean_squared_error', metrics=['mae'])


model.fit(X_train, y_train, batch_size=256, epochs=100, verbose=1)


test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Test MAE: {test_mae:.4f}")


model.save("battery_soc_model.h5")

model = tf.keras.models.load_model("battery_soc_model.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("battery_soc_model.tflite", "wb") as f:
    f.write(tflite_model)


Preprocessing complete. Data saved to preprocessed_data.csv.
Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


159/159 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.0390 - mae: 0.1266
Epoch 2/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0012 - mae: 0.0248
Epoch 3/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0010 - mae: 0.0230
Epoch 4/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 9.9113e-04 - mae: 0.0228
Epoch 5/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 9.7236e-04 - mae: 0.0227
Epoch 6/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 9.3382e-04 - mae: 0.0226
Epoch 7/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 9.0357e-04 - mae: 0.0221
Epoch 8/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 8.8822e-04 - mae: 0.0219
Epoch 9/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 8.8197e-04 - mae: 0.0218
Epoch 10/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 8.6176e-04 - mae: 0.0215
Epoch 11/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 8.1774e-04 - mae: 0.0207
Epoch 12/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 8.3989e-04 - m

Test MAE: 0.0137
Saved artifact at '/tmp/tmppy8ddg6x'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 3), dtype=tf.float32, name='input_layer_4')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  132494051786384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132494051837520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132494051785424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132494051776208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132494051838096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132494051835984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132494051836368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132494051839632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132494051838480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132494051840592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132494